In [1]:
!pip install -i https://test.pypi.org/simple/ LDVisualization-Alan1321==0.0.2 

Looking in indexes: https://test.pypi.org/simple/


In [2]:
from LDVisualization.inputHandlers import drop_down

In [3]:
drop_down()

Select a dataset from the options below: 

Dropdown(options=('TRMM_LIS_FULL', 'TRMM_LIS_SEASONAL', 'TRMM_LIS_MONTHLY', 'TRMM_LIS_DIURNAL', 'TRMM_LIS_DAIL…

Output()